# Import main calculation function


In [1]:
from geopolrisk.assessment.main import gprs_calc

Reading table/s ['Aluminium', 'Antimony', 'Arsenic', 'Asbestos', 'Baryte', 'Bauxite', 'Bentonite', 'Beryllium (conc.)', 'Bismuth', 'Boron Minerals', 'Cadmium', 'Chromium (Cr2O3)', 'Cobalt', 'Coking Coal', 'Copper', 'Diamonds (Gem)', 'Diamonds (Ind)', 'Diatomite', 'Feldspar', 'Fluorspar', 'Gallium', 'Germanium', 'Gold', 'Graphite', 'Gypsum and Anhydrite', 'Indium', 'Iron (Fe)', 'Kaolin (China-Clay)', 'Lead', 'Lignite', 'Lithium (Li2O)', 'logging', 'Magnesite', 'Manganese', 'Mercury', 'Molybdenum', 'Natural Gas', 'Nickel', 'Niobium (Nb2O5)', 'Oil Sands (part of Petroleum)', 'Oil Shales', 'Palladium', 'Perlite', 'Petroleum', 'Phosphate Rock (P2O5)', 'Platinum', 'Potash (K2O)', 'Rare Earths (REO)', 'Rhenium', 'Rhodium', 'Salt (rock, brines, marine)', 'Selenium', 'Silver', 'Steam Coal', 'Sulfur (elementar & industrial)', 'Talc, Steatite & Pyrophyllite', 'Tantalum (Ta2O5)', 'Tellurium', 'Tin', 'Titanium (TiO2)', 'Tungsten (W)', 'Uranium (U3O8)', 'Vanadium (V)', 'Vermiculite', 'Zinc', 'Zircon

## Prepare the lists


In [2]:
from geopolrisk.assessment.utils import default_rmlist

ListofMetals = default_rmlist()
ListofCountries = [156, 251, 842, "EU"]
ListofYear = [2022]

In [3]:
gprs_calc(ListofYear, ListofCountries, ListofMetals)

Calculating the GeoPolRisk: 100%|██████████| 212/212 [00:56<00:00,  3.76it/s]


,DBID,Country [Economic Entity],Raw Material,Year,GeoPolRisk Score,GeoPolRisk Characterization Factor [eq. Kg-Cu/Kg],HHI,Import Risk,Price
0,Aluminium1562022,China,Aluminium,2022,0.003945,0.024357,0.353279,0.011166,2.527817
1,Antimony1562022,China,Antimony,2022,0.076128,0.722417,0.276644,0.275184,3.885127
2,Arsenic1562022,China,Arsenic,2022,0.000152,0.075353,0.393237,0.000387,202.750097
3,Asbestos1562022,China,Asbestos,2022,0.108588,0.081655,0.334918,0.324222,0.307868
4,Baryte1562022,China,Baryte,2022,0.001981,0.000536,0.151374,0.013085,0.110799
...,...,...,...,...,...,...,...,...,...
207,TungstenEU2022,EU,Tungsten,2022,0.071131,4.342407,0.600702,0.118412,24.994016
208,UraniumEU2022,EU,Uranium,2022,0.153928,33.051142,0.247032,0.623111,87.908247
209,VanadiumEU2022,EU,Vanadium,2022,0.181131,8.651404,0.502120,0.360732,19.554918
210,ZincEU2022,EU,Zinc,2022,0.031120,0.096968,0.138882,0.224075,1.275700


In [4]:
from geopolrisk.assessment.core import HHI, importrisk, importrisk_company, GeoPolRisk
from geopolrisk.assessment.utils import mapped_baci

mapped_baci = mapped_baci()
resource = "Copper"
year = 2020
country = "China"
numerator, total_trade, price = importrisk(resource, year, country, data=mapped_baci)

# Testing individual functions for the same list


In [5]:
FirstMetal = ListofMetals[0]
FirstCountry = ListofCountries[0]
FirstYear = ListofYear[0]

In [ ]:
from geopolrisk.assessment.core import HHI


ProductionQuantity, hhi = HHI("Aluminium", 2021, FirstCountry)

In [7]:
df_query = f"(period == '{FirstYear}') & (reporterCode == {FirstCountry}) & (rawMaterial == '{FirstMetal}')"
baci_data = mapped_baci.query(df_query)

In [8]:
from geopolrisk.assessment.core import importrisk

Numerator, TotalTrade, Price = importrisk(
    FirstMetal, FirstYear, FirstCountry, data=mapped_baci
)  # Country has to be in a list

In [9]:
from geopolrisk.assessment.core import GeoPolRisk


Values = GeoPolRisk(Numerator, TotalTrade, Price, ProductionQuantity, hhi)

# Testing company level functions using same data as above


In [10]:
from geopolrisk.assessment.core import importrisk_company

Numerator_C, TotalTrade_C, Price_C = importrisk_company("Nickel", 2022)
Values_c = GeoPolRisk(
    Numerator_C, TotalTrade_C, Price_C, ProductionQuantity * 1000, hhi
)

  rawMaterial  partnerDesc    qty  cifvalue  period  Notes  partnerISO  \
0      Nickel      Denmark  170.0    3196.0    2022    NaN         208   
1      Nickel  Netherlands    1.0      73.0    2022    NaN         528   
2      Nickel     Portugal  893.0    2782.0    2022    NaN         620   
3      Nickel        Egypt    4.0     113.0    2022    NaN         818   
4      Nickel          USA  498.0    3355.0    2022    NaN         842   

   partnerWGI cmdCode reporterDesc  reporterISO  
0    0.326998  Nickel      Company          999  
1    0.356539  Nickel      Company          999  
2    0.327856  Nickel      Company          999  
3    0.705611  Nickel      Company          999  
4    0.507201  Nickel      Company          999  


# Testing Regional Data


In [11]:
NewRegions = {
    "EFTA": ["Iceland", "Norway", "Switzerland"],
    "NAFTA": ["Canada", "Mexico", "USA"],
}  # Liechtenstein has a customs union with Switzerland whose trade is not reported by UN Comtrade

In [12]:
from geopolrisk.assessment.utils import regions

regions(NewRegions)

In [13]:
from geopolrisk.assessment.database import databases

databases.regionslist

{'EU': ['Austria',
  'Belgium',
  'Bulgaria',
  'Croatia',
  'Cyprus',
  'Czechia',
  'Denmark',
  'Estonia',
  'Finland',
  'France',
  'Germany',
  'Greece',
  'Hungary',
  'Ireland',
  'Italy',
  'Latvia',
  'Lithuania',
  'Luxembourg',
  'Malta',
  'Netherlands',
  'Poland',
  'Portugal',
  'Romania',
  'Slovakia',
  'Slovenia',
  'Spain',
  'Sweden'],
 'Afghanistan': ['Afghanistan'],
 'Albania': ['Albania'],
 'Algeria': ['Algeria'],
 'Andorra': ['Andorra'],
 'Angola': ['Angola'],
 'Anguilla': ['Anguilla'],
 'Antigua and Barbuda': ['Antigua and Barbuda'],
 'Argentina': ['Argentina'],
 'Armenia': ['Armenia'],
 'Aruba': ['Aruba'],
 'Australia': ['Australia'],
 'Austria': ['Austria'],
 'Azerbaijan': ['Azerbaijan'],
 'Bahamas': ['Bahamas'],
 'Bahrain': ['Bahrain'],
 'Bangladesh': ['Bangladesh'],
 'Barbados': ['Barbados'],
 'Belarus': ['Belarus'],
 'Belgium': ['Belgium'],
 'Belgium-Luxembourg': ['Belgium-Luxembourg'],
 'Belize': ['Belize'],
 'Benin': ['Benin'],
 'Bermuda': ['Bermuda'],


In [14]:
databases.regional

True

## Main Function to run for the regional assessment


In [15]:
ListofMetals = [
    260400,
    261000,
    260300,
    283691,
    282200,
]
ListofCountries = [
    36,
    124,
    251,
    276,
    392,
    826,
    842,
    "EU",
    "EFTA",
    "NAFTA",
]  # EU is already defined in the library
ListofYear = [2018, 2019, 2020]
gprs_calc(ListofYear, ListofCountries, ListofMetals)

Calculating the GeoPolRisk:   0%|          | 0/150 [00:00<?, ?it/s]

Entered raw material does not exist in our database!


,DBID,Country [Economic Entity],Raw Material,Year,GeoPolRisk Score,GeoPolRisk Characterization Factor [eq. Kg-Cu/Kg],HHI,Import Risk,Price
